<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/basic_examples/getting_segment_bearings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting bearings and angles of channel segment

Last update by Simon M Mudd on 26/06/2023

This notebook shows how to extract the bearings and gradients of all channel segments. A segment is defined as a section of the river network between junctions.

## First install packages in colab

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this.
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **First get data** section.

First we install `lsdviztools`. This will take around a minute. It is important you do this before the `condacolab` step.

In [ ]:
!pip install lsdviztools &> /dev/null

Now we need to install lsdtopotools. We do this using something called `mamba`. To get `mamba` we install something called `condacolab`.

Note that `condacolab` restarts the kernal. Google colab will give you a "this session crashed" message but that is actually just `condacolab` restarting the session.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Now use mamba to install `lsdtopotools`.
This step takes a bit over a minute.

In [ ]:
!mamba install -y lsdtopotools &> /dev/null

This notebooks was made with `lsdtopotools` version 0.9. You can check the version with the following line:

In [ ]:
!lsdtt-basic-metrics -v

## First get data

We need to get some data to download.

For this example we will work in the Appalacian mountains in West Virginia, since this place has some quite interesting strucutral controls on channels.

We are going to download data using the opentopography scraper that is included with `lsdviztools`. You will need to get an opentopography.org account and copy in your API key.

You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "WV"
source_name = "COP30"

Xian_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [38.099764296412914, -80.76323669516319],
                        upper_right_coordinates = [39.24905595105265, -79.01641058948063],
                        prefix = Dataset_prefix,
                        api_key_file = your_OT_api_key_file)
Xian_DEM.print_parameters()
Xian_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

## Look at the data

Now we can just look at that data. First run a hillshade.

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"write_hillshade" : "true"}
Dataset_prefix = "WV"
source_name = "COP30"
r_prefix = Dataset_prefix+"_"+source_name+"_UTM"
w_prefix = Dataset_prefix+"_"+source_name+"_UTM"

lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Now plot the data with the next two code blocks.

In [ ]:
%%capture
# Use the capture comment to get rid of all the text
# But it will also not display an inline image
# So you need to call the image from the next line of code
DataDirectory = "./"
Base_file = r_prefix
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth",
                                 save_fig=True, size_format="geomorphology",dpi=600)

In [ ]:
print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

## Now get the segment information

To get the segment information, we use the switch `print_segment_bearings_and_gradients_to_csv`. This involves extracting a flow network. In order to not overwhelm the computer, we will set a rather large threshold for channel initiation. We set this with the `threshold_contributing_pixels` keyword. We also set a `SA_vertical_interval`. This is inherited from slope-area analysis: it calculates the gradient of the channel after a channel has fallen a vertical interval determined by this parameter (here 10m).  

I also `convert_csv_to_geojson` since this will make it a bit easier to look at the data.

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"print_segment_bearings_and_gradients_to_csv" : "true",
                    "SA_vertical_interval" : "10",
                    "print_channels_to_csv" : "true",
                    "threshold_contributing_pixels" : "5000",
                    "convert_csv_to_geojson" : "true"}

#}
Dataset_prefix = "WV"
source_name = "COP30"
r_prefix = Dataset_prefix+"_"+source_name+"_UTM"
w_prefix = Dataset_prefix+"_"+source_name+"_UTM"

lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This will create a `csv` file that has `segment_bearings` in the filename. We can have a look at this file:

In [ ]:
!ls *segment_bearings*

In [ ]:
import pandas as pd
df = pd.read_csv("WV_COP30_UTM_segment_bearings.csv")
df.head()

We can also look at the channel network

In [ ]:
import pandas as pd
df = pd.read_csv("WV_COP30_UTM_CN.csv")
df.head()

## Plot the bearings

First lets see where the channel network is:

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw
DataDirectory = "./"
Dataset_prefix = "WV"
source_name = "COP30"
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

#%%capture
channel_network_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, cmap = "jet", points_fname = "WV_COP30_UTM_CN.csv",
                             column_for_plotting = "Stream Order",
                             scale_points = True, column_for_scaling = "Stream Order",
                             scaled_data_in_log = True, max_point_size = 4, min_point_size = 1,
                             size_format = "ESURF",
                             fig_format = "png", dpi = 500, out_fname_prefix = "", save_fig = True)

Now lets try to plot:

In [ ]:
print(channel_network_img)
from IPython.display import display, Image
display(Image(filename=channel_network_img, width=800))

And now the channel bearings:

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw
DataDirectory = "./"
Dataset_prefix = "WV"
source_name = "COP30"
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

#%%capture
channel_bearing_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, cmap = "jet", points_fname = "WV_COP30_UTM_segment_bearings.csv",
                             column_for_plotting = "segment_bearing_whole_segment",
                             scale_points = True, column_for_scaling = "Stream Order",
                             scaled_data_in_log = True, max_point_size = 4, min_point_size = 1,
                             size_format = "ESURF",
                             fig_format = "png", dpi = 500, out_fname_prefix = "", save_fig = True)

In [ ]:
print(channel_bearing_img)
from IPython.display import display, Image
display(Image(filename=channel_bearing_img, width=800))